#### 1. Import & cấu hình

In [1]:
import os
import mysql.connector
import pandas as pd
from dotenv import load_dotenv

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np


#### 2. Load thông tin kết nối DB

In [2]:
load_dotenv()

MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", "3306"))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "")
MYSQL_DB = os.getenv("MYSQL_DB", "bookstore")

print(MYSQL_HOST, MYSQL_PORT, MYSQL_USER, MYSQL_DB)


localhost 3306 root bookstore


#### 3. Hàm connect MySQL

In [3]:
def get_connection():
    conn = mysql.connector.connect(
        host=MYSQL_HOST,
        port=MYSQL_PORT,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DB,
    )
    return conn

# test connect
conn = get_connection()
print("Connected:", conn.is_connected())
conn.close()


Connected: True


#### 4. Lấy dữ liệu sách từ DB
##### 4.1. Query books + author + categories

In [4]:
conn = get_connection()

query = """
SELECT 
    b.book_id,
    b.title,
    b.description,
    a.author_name,
    GROUP_CONCAT(DISTINCT c.category_name ORDER BY c.category_name SEPARATOR ' ') AS categories
FROM books b
LEFT JOIN authors a ON b.author_id = a.author_id
LEFT JOIN book_categories bc ON bc.book_id = b.book_id
LEFT JOIN categories c ON c.category_id = bc.category_id
WHERE b.status = 'active'
  AND b.stock_quantity > 0
GROUP BY b.book_id, b.title, b.description, a.author_name
"""

df_books = pd.read_sql(query, conn)
conn.close()

df_books.head()


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22912\1270759180.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_books = pd.read_sql(query, conn)


,book_id,title,description,author_name,categories
0,1,"""Học Công Nghệ Thông Tin, Rồi Làm Gì?""",Bạn đang băn khoăn: Học Công Nghệ Thông Tin th...,Vivian Vu,Công nghệ - Kỹ thuật
1,2,"""Học Quản Trị Kinh Doanh – Rồi Làm Gì?""","Bạn học ngành Quản trị kinh doanh (QTKD), nhưn...",Vivian Vu,Kinh doanh - Kinh tế
2,3,10 Bí Mật Khởi Nghiệp Bạn Cần Biết Trước Tuổi 30,Tôi vừa hoàn thành cuốn sách “10 Bí Mật Khởi N...,Zenson Tran,Kinh doanh - Kinh tế
3,4,10 Nghề Triển Vọng Không Thể Thay Thế Bởi AI,"Dưới đây là bài viết cảm nhận chân thật, đầy c...",Zenson Tran,Công nghệ - Kỹ thuật
4,5,10 Tập Nghìn Lẻ Một Đêm,Tóm tắt & Review (Đánh Giá) tiểu thuyết Nghìn ...,Antonie Galland,Văn học - Tiểu thuyết


##### 4.2. Tạo cột text tổng hợp

In [5]:
def safe_fill(x):
    return "" if x is None else str(x)

for col in ["title", "description", "author_name", "categories"]:
    df_books[col] = df_books[col].fillna("")

df_books["full_text"] = (
    df_books["title"] + " "
    + df_books["author_name"] + " "
    + df_books["categories"] + " "
    + df_books["description"]
)

df_books[["book_id", "title", "author_name", "categories", "full_text"]].head()


,book_id,title,author_name,categories,full_text
0,1,"""Học Công Nghệ Thông Tin, Rồi Làm Gì?""",Vivian Vu,Công nghệ - Kỹ thuật,"""Học Công Nghệ Thông Tin, Rồi Làm Gì?"" Vivian ..."
1,2,"""Học Quản Trị Kinh Doanh – Rồi Làm Gì?""",Vivian Vu,Kinh doanh - Kinh tế,"""Học Quản Trị Kinh Doanh – Rồi Làm Gì?"" Vivian..."
2,3,10 Bí Mật Khởi Nghiệp Bạn Cần Biết Trước Tuổi 30,Zenson Tran,Kinh doanh - Kinh tế,10 Bí Mật Khởi Nghiệp Bạn Cần Biết Trước Tuổi ...
3,4,10 Nghề Triển Vọng Không Thể Thay Thế Bởi AI,Zenson Tran,Công nghệ - Kỹ thuật,10 Nghề Triển Vọng Không Thể Thay Thế Bởi AI Z...
4,5,10 Tập Nghìn Lẻ Một Đêm,Antonie Galland,Văn học - Tiểu thuyết,10 Tập Nghìn Lẻ Một Đêm Antonie Galland Văn họ...


#### 5. Xây TF-IDF & similarity
##### 5.1. Tạo TF-IDF matrix (Term Frequency – Inverse Document Frequency)

TF (Term Frequency) – Độ “thường xuyên” của từ trong 1 tài liệu

IDF (Inverse Document Frequency) – Độ “đặc biệt” của từ trong toàn bộ tập tài liệu

TF-IDF = TF × IDF


TF-IDF là một phương pháp vector hóa / biểu diễn văn bản,
chứ không phải mô hình học máy “học tham số” như logistic regression, neural network, v.v.

Nó không học từ nhãn (label), chỉ tính toán dựa trên thống kê từ vựng → thuộc dạng feature extraction (trích xuất đặc trưng).

In [6]:
vectorlizer = TfidfVectorizer(
    stop_words=None,
    max_features=5000,
)


tftidf_matrix = vectorlizer.fit_transform(df_books["full_text"])
tftidf_matrix.shape

(3363, 5000)

##### 5.2. Tính cosine similarity

In [7]:
similarity_matrix = cosine_similarity(tftidf_matrix, tftidf_matrix)
similarity_matrix.shape

(3363, 3363)

##### 5.3. Hàm tiện test: tìm sách giống theo book_id / title

In [9]:
book_ids = df_books["book_id"].values
id_to_index = {book_id: idx for idx, book_id in enumerate(book_ids)}

def show_similar_by_book_id(p_book_id: int, top_n: int = 5):
    if p_book_id not in id_to_index:
        print("book_id", p_book_id, "không tồn tại trong df_books.")
        return
    idx = id_to_index[p_book_id]
    sims = list(enumerate(similarity_matrix[idx]))
    sims = sorted(sims, key=lambda x: x[1], reverse=True)
    sims = [s for s in sims if s[0] != idx][:top_n]
    
    print("Sách gốc:", df_books.loc[idx, "title"])
    print("Gợi ý tương tự:")
    for j, score in sims:
        print(f"- ({score:.4f})", df_books.loc[j, "book_id"], df_books.loc[j, "title"])

def show_similar_by_title(keyword: str, top_n: int = 5):
    found = df_books[df_books["title"].str.contains(keyword, case=False, na=False)]
    if found.empty:
        print("Không tìm thấy sách có title chứa:", keyword)
        return
    p_book_id = int(found.iloc[0]["book_id"])
    show_similar_by_book_id(p_book_id, top_n)


In [10]:
show_similar_by_book_id(1, top_n=10)

Sách gốc: "Học Công Nghệ Thông Tin, Rồi Làm Gì?"
Gợi ý tương tự:
- (0.5863) 1658 Ngành IT Có Gì? – Spiderum
- (0.4620) 1660 Ngành Sáng Tạo Và Nghệ Thuật Có Gì?
- (0.4298) 1000 HỌC NÔNG NGHIỆP – CÔNG NGHỆ CAO, RỒI LÀM GÌ?
- (0.4000) 1001 HỌC QUẢN LÝ TÀI NGUYÊN – MÔI TRƯỜNG, RỒI LÀM GÌ?
- (0.3990) 2 "Học Quản Trị Kinh Doanh – Rồi Làm Gì?"
- (0.3811) 2015 Polimedia
- (0.3801) 1002 HỌC TÂM LÝ HỌC, RỒI LÀM GÌ?
- (0.3769) 1087 Khoa học và công nghệ Việt Nam
- (0.3579) 2393 Thông tin và truyền thông
- (0.3478) 379 Chuyẻ̂n giao công nghệ ở Việt Nam


#### 6. Ghi kết quả vào bảng similar_books

In [11]:
N_SIMILAR = 20  # mỗi sách lưu top 20 cuốn giống nhất

conn = get_connection()
cursor = conn.cursor()

# Xóa dữ liệu cũ của TFIDF
cursor.execute("DELETE FROM similar_books WHERE algo_type = 'TFIDF'")
conn.commit()

rows_to_insert = []

for i, book_id in enumerate(book_ids):
    sims = list(enumerate(similarity_matrix[i]))
    sims = sorted(sims, key=lambda x: x[1], reverse=True)
    sims = [s for s in sims if s[0] != i][:N_SIMILAR]
    
    for j, score in sims:
        similar_book_id = int(book_ids[j])
        rows_to_insert.append((int(book_id), similar_book_id, float(score), 'TFIDF'))

insert_sql = """
INSERT INTO similar_books (book_id, similar_book_id, score, algo_type)
VALUES (%s, %s, %s, %s)
"""

cursor.executemany(insert_sql, rows_to_insert)
conn.commit()

cursor.close()
conn.close()

len(rows_to_insert)

67260

In [12]:
conn = get_connection()
df_sim = pd.read_sql("""
    SELECT sb.book_id, sb.similar_book_id, sb.score, b.title AS similar_title
    FROM similar_books sb
    JOIN books b ON b.book_id = sb.similar_book_id
    ORDER BY sb.book_id, sb.score DESC
    LIMIT 20
""", conn)
conn.close()

df_sim


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22912\474462913.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sim = pd.read_sql("""


,book_id,similar_book_id,score,similar_title
0,1,1658,0.5863,Ngành IT Có Gì? – Spiderum
1,1,1660,0.4620,Ngành Sáng Tạo Và Nghệ Thuật Có Gì?
2,1,1000,0.4298,"HỌC NÔNG NGHIỆP – CÔNG NGHỆ CAO, RỒI LÀM GÌ?"
3,1,1001,0.4000,"HỌC QUẢN LÝ TÀI NGUYÊN – MÔI TRƯỜNG, RỒI LÀM GÌ?"
4,1,2,0.3990,"""Học Quản Trị Kinh Doanh – Rồi Làm Gì?"""
5,1,2015,0.3811,Polimedia
6,1,1002,0.3801,"HỌC TÂM LÝ HỌC, RỒI LÀM GÌ?"
7,1,1087,0.3769,Khoa học và công nghệ Việt Nam
8,1,2393,0.3579,Thông tin và truyền thông
9,1,379,0.3478,Chuyẻ̂n giao công nghệ ở Việt Nam
